In [0]:
%sql
-- Pick any UC catalog/schema you have access to:
CREATE SCHEMA IF NOT EXISTS main.demo;

CREATE OR REPLACE TABLE main.demo.sales_daily (
  sale_date   DATE,
  store_id    STRING,
  product_id  STRING,
  sales_amount DOUBLE
);

INSERT OVERWRITE main.demo.sales_daily VALUES
  (DATE'2026-01-01', 'store_1', 'A', 10.0),
  (DATE'2026-01-02', 'store_1', 'A', 20.0),
  (DATE'2026-01-03', 'store_1', 'A', 30.0),
  (DATE'2026-01-04', 'store_1', 'A', 40.0),
  (DATE'2026-01-05', 'store_1', 'A', 50.0),
  (DATE'2026-01-06', 'store_1', 'A', 60.0),
  (DATE'2026-01-07', 'store_1', 'A', 70.0),
  (DATE'2026-01-08', 'store_1', 'A', 80.0);


In [0]:
CREATE OR REPLACE VIEW main.demo.sales_metric_view
WITH METRICS
LANGUAGE YAML
AS $$
version: 1.1
comment: "Daily revenue + 7-day moving average using SQL window functions"

source: main.demo.sales_daily

dimensions:
  - name: sale_date
    expr: sale_date
  - name: store_id
    expr: store_id
  - name: product_id
    expr: product_id

measures:
  - name: daily_revenue
    expr: SUM(sales_amount)

  - name: revenue_ma_7d
    expr: |
      AVG(SUM(sales_amount)) OVER (
        PARTITION BY store_id, product_id
        ORDER BY sale_date
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
      )
$$;


In [0]:
SELECT
  sale_date,
  store_id,
  product_id,
  MEASURE(daily_revenue)  AS daily_revenue,
  MEASURE(revenue_ma_7d)  AS revenue_ma_7d
FROM main.demo.sales_metric_view
-- WHERE store_id = 'store_1'
--   AND product_id = 'A'
-- GROUP BY ALL
-- ORDER BY sale_date;